In [ ]:
# Подгрузка библиотек

In [ ]:
import csv
import os
import numpy as np
import pandas as pd
from collections import Counter
import re

import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

import warnings

warnings.filterwarnings('ignore')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# df_path = '/content/drive/MyDrive/hacaton_1/SERGEY'

In [ ]:
df_path = './data-copy'

In [ ]:
tqdm.pandas(desc="Processing rows...", ncols=100)

# 1. Загрузка данных

Исходный набор представляет собой открытые данные [Государственного каталога музейного фонда РФ Министерства культуры РФ](https://opendata.mkrf.ru/opendata/7705851331-museum-exhibits)

In [ ]:
# Подгрузка данных
data = pd.read_csv(f'{df_path}/Russian_museum.csv', sep=';', encoding='utf-8')
data.head()

In [ ]:
data['Автор'].value_counts()

In [ ]:
def extract_surname(author: str) -> str:
    if not isinstance(author, str) or not author.strip():
        return "Неизвестный"
    # Убираем кавычки и квадратные скобки
    author = re.sub(r'^[\[\]"]+|[\[\]"]+$', '', author)
    # Убираем даты (разные форматы записи)
    author = re.sub(r'\d{4}([–—,\(\)]*\d{4})?', '', author)
    # Убираем инициалы
    author = re.sub(r'\b[А-ЯЁ]\.? ?[А-ЯЁ]\.? ?', '', author)
    # Убираем ненужные слова: "г.", "род.", "ум.", "по проекту", "оглы" и т. д.
    author = re.sub(r'\b(г\.|род\.|ум\.|по|проекту|оглы|фон|де|мастер|скульптор|гравёр|художник|автор)\b', '', author, flags=re.IGNORECASE)
    # Убираем оставшиеся символы пунктуации
    author = re.sub(r'[^А-Яа-яЁё-]', ' ', author)
    # Убираем лишние пробелы
    author = re.sub(r'\s+', ' ', author).strip()
    # Если после очистки ничего не осталось, возвращаем "Неизвестный"
    if not author:
        return "Неизвестный"
    # Оставляем только первую часть имени (фамилию)
    surname = author.split()[0]
    return surname

In [ ]:
# Применяем функцию
data['Фамилия'] = data['Автор'].progress_apply(extract_surname)

data['Фамилия'].value_counts()

# 2. Очистка данных

In [ ]:
# Удаление дубликатов
data_cleaned = data.drop_duplicates()

# Сброс индексов после удаления дубликатов
data_cleaned = data_cleaned.reset_index(drop=True)

In [ ]:
data_cleaned

In [ ]:
def get_graph(top_5, column):
    plt.figure(figsize=(8, 6))
    sns.barplot(x=top_5.index, y=top_5.values, palette='viridis')
    plt.title(f'Top 5 Frequent Values in {column}')
    plt.xlabel('Category')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
 for column in data_cleaned.columns:
    print(column)
    if data[column].dtype == 'object' or data[column].dtype.name == 'category':
        # Получаем топ-5 частых значений для каждого категориального признака
        top_5 = data_cleaned[column].value_counts().head(5)
        get_graph(top_5, column)


In [ ]:
data_cleaned.info()

In [ ]:
# Исключаем неинформативные признаки
cols_for_drop = ['Музей', 'Строковое описание размеров', 'Количество составляющих', 'Инвентарный номер', 'Номер по ГИК',
                 'Ширина', 'Длина', 'Единица измерения размера', 'Место создания', 'Количество составляющих',
                 'Ключевые слова', 'URL предмета на сайте музея', 'Высота', 'Идентификатор статуса предмета',
                 'Краткое описание истории бытования (провенанс) предмета',
                 'Точность задания времени (день, месяц, год, век, эпоха).',
                 'Интервал времени создания предмета (начало)',
                 'Интервал времени создания предмета (окончание)',

                 ]
data_cleaned.drop(cols_for_drop, axis=1, inplace=True)

In [ ]:
data_cleaned

In [ ]:
data_cleaned.columns

In [ ]:
# Переименовываем столбцы
data_cleaned = data_cleaned.rename(columns={
    'Наименование предмета': 'title',
    'Автор': 'author_full_name',
    'Фамилия': 'author',
    'Items': 'items',
    'Описание': 'description',
    'Изображение': 'image_url',
    'Регистрационный номер Госкаталога': 'catalog_num',
    'Дата регистрации записи.': 'registration_date',
    'Типология': 'typology',
    'Дата создания предмета (строка)': 'creation_date',

})

In [ ]:
data_cleaned.columns

## Выбор графики и живописи

In [ ]:
data_cleaned['typology'].value_counts()

In [ ]:
# Приводим столбец 'Типология' к нижнему регистру и удаляем лишние пробелы
data_cleaned['typology'] = data_cleaned['typology'].str.strip().str.lower()

# Выбираем строки с типологией 'графика' или 'живопись'
picture_data = data_cleaned[data_cleaned['typology'].isin(['графика', 'живопись'])].reset_index(drop=True)

In [ ]:
picture_data

In [ ]:
# Выбор данных, которые не относятся к 'графика' и 'живопись'
other_data = data_cleaned[~data_cleaned['typology'].isin(['графика', 'живопись'])]
other_data.to_csv(f'{df_path}/other_data.csv', index=False)

## Очистка столбца Материалы

In [ ]:
picture_data['items'] = picture_data['items'].str.replace('["', '').str.replace('"]', '')

In [ ]:
picture_data['items']

In [ ]:
synonyms = {
    r'^графитный карандаш$': 'карандаш',
    r'^цветной карандаш$': 'карандаш',
    r'^карандаш графитовый$': 'карандаш',
    r'^карандаш графитный$': 'карандаш',
    r'^цифровая печать$': 'печать',
    r'^типографская печать$': 'печать',
    r'^альбуминовая печать$': 'печать',
    r'^лит$': 'литография',
    r'^литография с тоном$': 'литография',
    r'^литография цветная$': 'литография',
    r'^бумага верже$': 'бумага',
    r'^роспись полихромная$': 'роспись',
    r'^б.$': 'бумага',
    r'^черные чернила$': 'тушь',
    r'^акв$': 'акварель',
    r'^акв\.$': 'акварель',
    r'^гравюра пунктиром$': 'гравюра',
    r'^гравюра на дереве$': 'гравюра',
    r'^карандаш итальянский$': 'карандаш',
    r'^гравюра резцом$': 'гравюра',
    r'^бумага на картоне$': 'бумага',
    r'^бумага серая$': 'бумага',
    r'^черный карандаш$': 'карандаш',
    r'^граф\. кар\.$': 'карандаш',
    r'^карандаши цветные$': 'карандаш',
    r'^карандаш черный$': 'карандаш',
    r'^акварель черная$': 'акварель',
    r'^линогравюра$': 'гравюра',
    r'^шариковая ручка$': 'чернила',
    r'^хромолитография$': 'литография',
    r'^автолитография$': 'литография',
    r'^литография раскрашенная$': 'литография',
    r'^фанера$': 'дерево',
    r'^акварельарель черная$': 'акварель',
    r'^цветная литография$': 'литография',
    r'^граф\. кар$': 'карандаш',
    r'^холст на картоне$': 'холст',
    r'^карандаш угольный$': 'карандаш',
    r'^бумага цветная$': 'бумага',
    r'^бумага желтая$': 'бумага',
    r'^бумага черная$': 'бумага',
    r'^черная акварель$': 'акварель',
    r'^акварель атинта$': 'акварель',
    r'^\\nмасло$': 'масло',
    r'^гравюра цветная$': 'гравюра',
    r'^итальянский карандаш$': 'карандаш',
    r'^гравюра раскрашенная$': 'гравюра',
    r'^монотипия цветная$': 'монотипия',
    r'^уголь прессованный$': 'уголь',
    r'^граф\.кар\.$': 'картон',
    r'^бумага желтоватая$': 'бумага',
    r'^бумага коричневая$': 'бумага',
    r'^цветные карандаши$': 'карандаш',
    r'^карандаш свинцовый$': 'карандаш',
    r'^холст,масло$': 'масло',
    r'^масляная пастель$': 'пастель',
    r'^наклеенная на белую бумагу$': 'аппликация',
    r'^наклейки$': 'аппликация',
    r'^бумага мелованная$': 'бумага',
    r'^ксилография цветная$': 'ксилография',
    r'^карандаш литографский$': 'литография',
    r'^тушь цветная$': 'тушь',
    r'^мягкий лак$': 'лак',
    r'^гравюра на картоне$': 'гравюра',
    r'^картон мелованный$': 'картон',
    r'^бронзовая краска$': 'бронза',
    r'^к\.$': 'картон',
    r'^бумага голубая$': 'бумага',
    r'^бумага оберточная$': 'бумага',
    r'^слоновая кость\\nминиатюра$': 'миниатюра',
    r'^автолитография цветная$': 'литография',
    r'^гравюра резцом раскрашенная$': 'гравюра',
    r'^линогравюра цветная$': 'гравюра',
    r'^черн. акв$': 'акварель',
    r'^холст,\\nмасло$': 'масло',
    r'^цв. кар$': 'картон',
    r'^бумага зеленая$': 'бумага'

}


In [ ]:
def clean_material(material, synonyms):
    if pd.isna(material):
        material = ''
    else:
        material = material.strip().lower()
    for old, new in synonyms.items():
        material = re.sub(old, new, material)
    return material

In [ ]:
def analyze_top_materials(dataframe, column_name, synonyms, top_n=50):
    all_materials = dataframe[column_name].str.split(', ').explode()
    all_materials_cleaned = all_materials.progress_apply(lambda material: clean_material(material, synonyms))
    material_counts = Counter(all_materials_cleaned)
    counter = pd.DataFrame.from_dict(material_counts, orient='index', columns=['Count'])
    counter_cleaned = counter.sort_values(by='Count', ascending=False)
    top_materials = counter_cleaned.head(top_n)

    return top_materials.index.tolist()


In [ ]:
top_50 = analyze_top_materials(picture_data, 'items', synonyms)
top_50

In [ ]:
# Функция для обработки строки
def process_items(item):
    if pd.isna(item):
        return ''
    # Разделяем строку на отдельные материалы
    materials = item.split(', ')
    # Заменяем синонимы
    materials = [material for material in materials if material in top_50]
    # Удаляем дубликаты и сортируем
    return ','.join(sorted(set(materials)))

In [ ]:
# Преобразуем столбец items в key_words
picture_data['key_materials'] = picture_data['items'].progress_apply(process_items)

In [ ]:
picture_data['key_materials']

In [ ]:
picture_data.to_csv(f'{df_path}/picture_data_with_key_material.csv', index=False, encoding='utf-8', sep=';')

## Очистка столбца с датами

In [ ]:
# функция для предобработки дат
def process_date(date_str):
    try:
        # Проверка на NaN
        if pd.isna(date_str):
            return "Неизвестно"

        # Преобразуем строку
        date_str = re.sub(r"[^\w\s\-]", "", date_str).lower()
        date_str = date_str.replace("х", "х годов")

        # Обработка "конец" (например, "конец 1920-х" или "конец 1900-х")
        if "конец" in date_str:
            decades = re.findall(r"\d{3,4}", date_str)
            if len(decades) == 1:
                start = int(decades[0])
                if start % 100 != 0:
                    return f"{start + 6}-{start + 9}"  # для конца десятилетия, например, конец 1920-х -> 1926-1929
                else:
                    return f"{start + 96}-{start + 99}"  # для конца десятилетия после 1900-х, например, конец 1900-х -> 1996-1999
            elif len(decades) == 2:
                return f"{decades[0]}-{decades[1]}"

        # Обработка "начало" (например, "начало 1920-х")
        if "начало" in date_str:
            decades = re.findall(r"\d{3,4}", date_str)
            if len(decades) == 1:
                start = int(decades[0])
                return f"{start}-{start + 9}"  # для начала десятилетия, например, начало 1920-х -> 1920-1929
            elif len(decades) == 2:
                return f"{decades[0]}-{decades[1]}"

        # Обработка "середина" (например, "середина 1920-х")
        if "середина" in date_str:
            decades = re.findall(r"\d{3,4}", date_str)
            if len(decades) == 1:
                start = int(decades[0])
                return f"{start + 4}-{start + 5}"

        # Обработка "1-я половина" и "2-я половина"
        if "1-я половина" in date_str or "1-ая половина" in date_str:
            decades = re.findall(r"\d{3,4}", date_str)
            if len(decades) == 1:
                start = int(decades[0])
                return f"{start}-{start + 49}"

        if "2-я половина" in date_str or "2-ая половина" in date_str:
            decades = re.findall(r"\d{3,4}", date_str)
            if len(decades) == 1:
                start = int(decades[0])
                return f"{start + 50}-{start + 99}"

        # Обработка диапазонов
        if "-" in date_str:
            years = re.findall(r"\d{4}", date_str)
            if len(years) == 2:
                start_year = int(years[0])
                end_year = int(years[1])
                if start_year >= 1000 and end_year <= 3000:
                    return f"{start_year}-{end_year}"
            elif len(years) == 1:
                return f"{years[0]}"

        # Обработка "около"
        if "ок" in date_str or "примерно" in date_str:
            years = re.findall(r"\d{4}", date_str)
            if len(years) == 1:
                return years[0]

        # Обработка одиночных годов
        years = re.findall(r"\d{4}", date_str)
        if len(years) == 1:
            return years[0]
        # Если ничего не найдено
        return "Неизвестно"
    except:
        return "Ошибка"


In [ ]:
# Обработка всех строк
picture_data['date_cleaned'] = picture_data['creation_date'].progress_apply(process_date)
picture_data[['date_cleaned', 'creation_date']]

In [ ]:
def categorize_date(date_str):
    try:
        # Проверка на NaN и ошибки
        if pd.isna(date_str) or date_str == "Неизвестно":
            return "Неизвестно"

        # Обработка диапазонов (например, 1920-1930)
        if "-" in date_str:
            years = re.findall(r"\d{4}", date_str)
            if len(years) == 2:
                start_year = int(years[0])
                end_year = int(years[1])
                avg_year = (start_year + end_year) // 2
            else:
                return "Неизвестно"
        else:
            avg_year = int(date_str)

        # Определяем век на основе среднего года
        century = (avg_year // 100) + 1

        # Определяем, первая или вторая половина века
        if avg_year % 100 <= 50:
            return f"1-я пол.{century}"
        else:
            return f"2-я пол.{century}"

    except:
        return f"Ошибка"

In [ ]:
picture_data['date_category'] = picture_data['date_cleaned'].progress_apply(categorize_date)
picture_data[['date_category', 'date_cleaned', 'creation_date']]

In [ ]:
picture_data.to_csv(f'{df_path}/picture_data_with_date_and_materials.csv', sep=';', encoding='utf-8', index=False)

# Поиск ключевых слов по названию и описанию

In [ ]:
import spacy
from tqdm.notebook import tqdm
from spacy.lang.ru.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
!python -m spacy download ru_core_news_sm

In [ ]:
# Загрузка модели spacy для русского языка
nlp = spacy.load("ru_core_news_sm")

In [ ]:
# Функция для лемматизации и очистки текста
def lemmatize_text_spacy(text):
    doc = nlp(text.lower())  # Приводим к нижнему регистру
    lemmatized_words = [
        token.lemma_ for token in doc
        if token.is_alpha  # Убираем числа и знаки препинания
        and len(token.lemma_) > 1  # Исключаем отдельные буквы
        and token.lemma_ not in STOP_WORDS  # Исключаем стоп-слова
    ]
    return ' '.join(lemmatized_words)

In [ ]:
def extract_keywords_with_tfidf(df, top_n=15):
    # Объединяем описание и название в единый текст и лемматизируем
    df['combined_text'] = df['description'] + ' ' + df['title']
    df['lemmatized_text'] = df['combined_text'].apply(lemmatize_text_spacy)

    # Инициализируем TF-IDF векторизатор с русскими стоп-словами
    vectorizer = TfidfVectorizer(stop_words=list(STOP_WORDS))
    tfidf_matrix = vectorizer.fit_transform(df['lemmatized_text'])
    feature_names = vectorizer.get_feature_names_out()

    df['key_words'] = ''

    for index in tqdm(range(len(df)), total=len(df), desc="Processing rows", ncols=100):
        # Получаем TF-IDF веса для текущей строки
        tfidf_vector = tfidf_matrix[index].toarray().flatten()
        top_indices = tfidf_vector.argsort()[-top_n:][::-1]

        # Извлекаем ключевые слова
        key_words = [feature_names[i] for i in top_indices if tfidf_vector[i] > 0]

        # Сохраняем ключевые слова
        df.at[index, 'key_words'] = ','.join(key_words)

    # Удаляем временные колонки
    df.drop(columns=['combined_text', 'lemmatized_text'], inplace=True)

    return df

In [ ]:
df = extract_keywords_with_tfidf(picture_data)

In [ ]:
# Сохранение результата в CSV файл
df.to_csv(f'{df_path}/picture_data_with_keywords.csv', index=False, sep=';', encoding='utf-8')

In [ ]:
df[['key_words', 'title', 'description']]

# Сохранение датафрейма для кластеризации и загрузка в бд

In [ ]:
df.drop(['date_cleaned', 'key_materials', 'creation_date'], inplace=True, axis=1)

In [ ]:
columns_for_db = [
    'image_url', 'catalog_num', 'registration_date',
]

In [ ]:
def reformate_url(column):
    if pd.isna(column):
        return None
    match = re.search(r'"url":"(http[s]?://[^"]+)"', column)
    if match:
        return match.group(1)
    return None

In [ ]:
df['image_url'] = df['image_url'].apply(reformate_url)

In [ ]:
df['image_url'].head()

In [ ]:
df.to_csv(f'{df_path}/data_for_database.csv', index=False, sep=';', encoding='utf-8')

In [ ]:
df = pd.read_csv(f'{df_path}/data_for_database.csv', sep=';', encoding='utf-8')

In [ ]:
df.drop(columns_for_db, inplace=True, axis=1)

In [ ]:
df.columns

In [ ]:
df.rename(columns={
    'author_lastname': 'author',
}, inplace=True)

In [ ]:
df.to_csv(f'{df_path}/data_for_clasterisation.csv', index=False, sep=';', encoding='utf-8')

In [ ]:
df

# Рекомендательная система

In [ ]:
import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from tqdm import tqdm
from transformers import BertTokenizer, BertModel


In [ ]:
folder_path = './data-copy'
df_path = folder_path + '/data_for_clasterisation.csv'

In [ ]:
# Инициализация tqdm_pandas
tqdm.pandas()

In [ ]:
df = pd.read_csv(df_path, sep=';', encoding='utf-8')
df

In [ ]:
# очистка пропусков
df['key_words'] = df['key_words'].fillna('')
df['author'] = df['author'].fillna('')
df['key_words'] = df['key_words'] + ',' + df['author']

## 2. Кодирование признаков

In [ ]:
# Загрузка предобученной модели для русского языка
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
# Функция для получения эмбеддингов с помощью RuBERT
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

In [ ]:
def get_item_embeddings(data):
    descriptions = data['key_words'].tolist()
    embeddings = []
    # Получаем эмбеддинги для каждого экспоната
    for desc in tqdm(descriptions, desc="Генерация эмбеддингов для описаний", unit="экспонат"):
        embedding = get_embeddings([desc])
        embeddings.append(embedding)
    # Преобразуем список эмбеддингов в numpy массив
    embeddings = torch.cat(embeddings, dim=0).numpy()
    return embeddings

In [ ]:
# Функция для дополнения эмбеддинга запроса до требуемой размерности
def resize_embedding(query_embedding, target_dim):
    current_dim = query_embedding.shape[1]

    # Если текущая размерность уже совпадает с целевой, возвращаем эмбеддинг без изменений
    if current_dim == target_dim:
        return query_embedding

    # Если текущая размерность меньше целевой, дополняем нулями
    if current_dim < target_dim:
        padding = np.zeros((query_embedding.shape[0], target_dim - current_dim))
        return np.hstack([query_embedding, padding])

    # Если текущая размерность больше целевой, обрезаем лишние элементы
    return query_embedding[:, :target_dim]


In [ ]:
# Функция визуализации кластеров
def visualize_clusters(features, data, method="PCA"):
    # Снижение размерности
    if method == "PCA":
        reducer = PCA(n_components=2)
    elif method == "TSNE":
        reducer = TSNE(n_components=2, random_state=42, perplexity=30)
    else:
        raise ValueError("Метод должен быть 'PCA' или 'TSNE'")

    reduced_features = reducer.fit_transform(features)

    # Создание DataFrame для визуализации
    visualization_df = pd.DataFrame(reduced_features, columns=["Dim1", "Dim2"])
    visualization_df['cluster'] = data['cluster']

    # Визуализация кластеров
    plt.figure(figsize=(10, 8))
    sns.scatterplot(
        x="Dim1", y="Dim2",
        hue="cluster",
        palette="tab10",
        data=visualization_df,
        legend="full"
    )
    plt.title(f"Визуализация кластеров ({method})", fontsize=16)
    plt.xlabel("Первая компонента", fontsize=12)
    plt.ylabel("Вторая компонента", fontsize=12)
    plt.legend(title="Кластеры")
    plt.show()


In [ ]:
from sklearn.metrics import silhouette_score

def plot_silhouette_score(X, min_clusters=2, max_clusters=10):
    scores = []
    cluster_range = range(min_clusters, max_clusters + 1)

    for k in cluster_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
        labels = kmeans.fit_predict(X)
        score = silhouette_score(X, labels)
        scores.append(score)

    # Построение графика
    plt.figure(figsize=(8, 5))
    plt.plot(cluster_range, scores, marker='o', linestyle='--')
    plt.xlabel("Число кластеров")
    plt.ylabel("Силуэтный коэффициент")
    plt.title("Определение оптимального числа кластеров")
    plt.xticks(cluster_range)
    plt.grid(True)
    plt.show()

In [ ]:
embeddings = get_item_embeddings(df)

In [ ]:
embeddings

In [ ]:
np.save(folder_path + '/embeddings.npy', embeddings)

In [ ]:
# Сохранение в CSV
with open(folder_path + 'embeddings.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(embeddings)

In [ ]:
embeddings.shape

In [ ]:
!pip install pyarrow

In [ ]:
df['embeddings'] = list(embeddings)
# Сохраняем в Parquet (меньше места, быстрее загрузка)
df.to_parquet(folder_path + "/dataset_with_embeddings.parquet", index=False, engine="pyarrow")

In [ ]:
df = pd.read_parquet(folder_path + "/dataset_with_embeddings.parquet")
df

## Рекомендатлеьная система

In [ ]:
embeddings = df['embeddings']
embeddings.dtype

In [ ]:
def create_kmeans_model(embeddings, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(embeddings)
    return kmeans

In [ ]:
plot_silhouette_score(embeddings)

In [ ]:
kmeans_model = create_kmeans_model(embeddings, n_clusters=5)
kmeans_model.cluster_centers_.shape

In [ ]:
from joblib import dump, load

# Сохранение модели
dump(kmeans_model, './data-copy/kmeans_model2.joblib')

In [ ]:
# Загрузка модели
kmeans_model = load('./data-copy/kmeans_model2.joblib')
kmeans_model.cluster_centers_.shape

In [ ]:
# Прогнозируем, к какому кластеру принадлежит каждый экспонат
cluster_labels = kmeans_model.predict(embeddings)

# Добавляем информацию о кластере в данные
df['cluster'] = cluster_labels

In [ ]:
# Визуализация кластеров с использованием PCA
visualize_clusters(embeddings, df, method="PCA")

In [ ]:
# Визуализация кластеров с использованием t-SNE
visualize_clusters(embeddings, df, method="TSNE")

In [ ]:
df.to_parquet(folder_path + 'data_with_clasters2.parquet', index=False, engine="pyarrow")

In [ ]:
df = pd.read_parquet(folder_path + 'data_with_clasters2.parquet')
df

In [ ]:
df_to_merge = pd.read_csv('./data/data_for_database.csv', sep=';', encoding='utf-8')

In [ ]:
df_to_merge

In [ ]:
df['author'].rename('author_lastname')
df['author_name'] = df_to_merge['author_name']
df['author_patronymic'] = df_to_merge['author_patronymic']
df['catalog_num'] = df_to_merge['catalog_num']
df['registration_date'] = df_to_merge['registration_date']
df['image_url'] = df_to_merge['image_url']
df["registration_date"] = pd.to_datetime(df["registration_date"])
df

In [ ]:
df.to_parquet(folder_path + '/data_for_database_final2.parquet', index=False, engine="pyarrow")

In [ ]:
df = pd.read_parquet(folder_path + '/data_for_database_final2.parquet')

## Сохранение в бд

In [ ]:
!pip install python-dotenv sqlalchemy psycopg2

In [ ]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy.orm import sessionmaker

# Загружаем переменные окружения из .env файла
load_dotenv()

# Получаем параметры из окружения
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')

# Формируем строку подключения для PostgreSQL
DATABASE_URL = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Создание подключения к базе данных
engine = create_engine(DATABASE_URL)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
df['embeddings'] = df['embeddings'].apply(lambda x: x.tolist())

In [ ]:
# Пример использования tqdm с to_sql
try:
    chunksize = 10  # Размер батча
    num_chunks = len(df) // chunksize + 1

    # Создаем прогресс-бар с tqdm
    with tqdm(total=num_chunks, desc="Загрузка в PostgreSQL") as pbar:
        for start in range(0, len(df), chunksize):
            df.iloc[start:start + chunksize].to_sql("images", engine, if_exists="append", index=False,
                                                    chunksize=chunksize)
            pbar.update(1)

    print("✅ Датасет загружен в PostgreSQL")
except Exception as e:
    session.rollback()
    print(f"❌ Ошибка: {e}")
finally:
    session.close()

## Рекомендательная система KMeans

In [ ]:
# Рекомендация экспонатов по кластеру
def recommend_by_cluster(query, data, kmeans_model, n_recommendations=10):
    # Получаем эмбеддинг запроса
    query_embedding = get_embeddings([query]).numpy()

    # Приводим эмбеддинг к типу float32
    query_embedding = query_embedding.astype(np.float32)

    # Определяем кластер для запроса
    query_cluster = kmeans_model.predict(query_embedding)

    # Рекомендуем экспонаты из того же кластера
    recommended_items = data[data['cluster'] == query_cluster[0]].head(n_recommendations)
    return recommended_items


In [ ]:
# Пример: Рекомендуем экспонаты для пользователя с запросом
user_query = "картина, изображающая пейзаж с озером"
recommended_items = recommend_by_cluster(user_query, df, kmeans_model)
recommended_items[['title', 'author', 'date_category']]

## Рекомендательная система на основе KMeans + KNN

In [ ]:
def recommend_by_kmeans_knn(query, data, embeddings, kmeans_model, n_recommendations=10):
    # Получаем целевую размерность из KMeans
    target_dim = kmeans_model.cluster_centers_.shape[1]

    # Получаем эмбеддинг запроса
    query_embedding = get_embeddings([query]).numpy()

    # Приводим эмбеддинг к нужной размерности
    query_embedding_resized = resize_embedding(query_embedding, target_dim)

    # Определяем кластер для запроса
    query_cluster = kmeans_model.predict(query_embedding_resized)

    # Получаем индексы объектов в этом кластере
    cluster_indices = np.where(kmeans_model.labels_ == query_cluster)[0]

    # Если embeddings - разреженная матрица, конвертируем в плотный массив
    if isinstance(embeddings, np.ndarray):
        cluster_embeddings = embeddings[cluster_indices]
    else:
        cluster_embeddings = np.asarray(embeddings.todense()[cluster_indices])

    # Создаем модель NearestNeighbors
    nn_model = NearestNeighbors(n_neighbors=n_recommendations, metric='cosine')
    nn_model.fit(cluster_embeddings)

    # Находим ближайших соседей для запроса в этом кластере
    distances, indices = nn_model.kneighbors(query_embedding_resized)

    # Возвращаем рекомендованные объекты
    recommended_items = data.iloc[cluster_indices[indices.flatten()]]

    return recommended_items


In [ ]:
# Пример: Рекомендуем экспонаты для пользователя с запросом
user_query = "картина, изображающая пейзаж с озером"

# Рекомендуем экспонаты, используя модель KMeans и KNN
recommended_items = recommend_by_kmeans_knn(user_query, df, embeddings, kmeans_model)

# Выводим рекомендованные экспонаты с нужными колонками
recommended_items[['title', 'author', 'date_category']]

# Функции для чат-бота

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from transformers import BertTokenizer, BertModel
import torch
from dotenv import load_dotenv
import os
import joblib  # Для загрузки k-means модели

# === Загрузка модели KMeans ===
kmeans_model = joblib.load(folder_path + "/kmeans_model.joblib")

# === Загрузка переменных окружения ===
load_dotenv()

db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')

DATABASE_URL = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# === Загрузка модели RuBERT ===
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()  # Переключаем в режим инференса

# === Загрузка эмбеддингов из parquet ===
df = pd.read_parquet(folder_path + '/data_for_database_final.parquet')
df["id"] = df.index

# Преобразуем эмбеддинги в numpy-массив
embeddings = np.vstack(df["embeddings"].apply(np.array))  # Если хранятся как списки


# === Функция для получения эмбеддингов запроса ===
def get_embeddings(query):
    inputs = tokenizer(query, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Усредняем по нужной размерности
    return embeddings.cpu().numpy()  # Переводим в numpy


# === Функция рекомендаций через KMeans + KNN ===
def recommend_by_kmeans_knn(query, embeddings, kmeans_model, n_recommendations=5):
    query_embedding = get_embeddings(query)  # Уже numpy
    query_cluster = kmeans_model.predict(query_embedding)[0]  # Получаем кластер
    cluster_indices = np.where(kmeans_model.labels_ == query_cluster)[0]  # Выбираем объекты из кластера

    cluster_embeddings = embeddings[cluster_indices]  # Берем их эмбеддинги

    # Создаем KNN-модель
    nn_model = NearestNeighbors(n_neighbors=n_recommendations, metric='cosine')
    nn_model.fit(cluster_embeddings)

    distances, indices = nn_model.kneighbors(query_embedding)

    recommended_indices = cluster_indices[indices.flatten()]  # Переводим в индексы исходного массива
    return recommended_indices


# === Функция получения экспонатов по индексам ===
def get_exhibits_by_indices(indices):
    return df.iloc[indices][["id", "title", "author", "date_category"]].to_dict(orient="records")


# === Пример использования ===
user_query = "картина, изображающая пейзаж с озером"
recommended_indices = recommend_by_kmeans_knn(user_query, embeddings, kmeans_model)

# Получаем рекомендованные экспонаты
recommended_exhibits = get_exhibits_by_indices(recommended_indices)
print(recommended_exhibits, sep='\n')


In [ ]:
 import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from transformers import BertTokenizer, BertModel
import torch
from dotenv import load_dotenv
import os
import ast
import joblib  # Для загрузки k-means модели
from sqlalchemy import create_engine

# === Загрузка модели KMeans ===
kmeans_model = joblib.load("./data-copy/kmeans_model.joblib")

# === Загрузка переменных окружения ===
load_dotenv()

db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')

DATABASE_URL = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Создаем движок для подключения к базе данных
engine = create_engine(DATABASE_URL)

# === Загрузка модели RuBERT ===
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


# === Загрузка эмбеддингов из базы данных ===
query = """SELECT  catalog_num, title, author, date_category, embeddings FROM images_1"""
df = pd.read_sql(query, engine)

# Проверка типов данных
print(df.dtypes)

# Преобразуем эмбеддинги в numpy-массив
# embeddings = np.vstack(df["embeddings"].apply(np.array))  # Предполагаем, что эмбеддинги хранятся как списки
# Функция для преобразования строк в массивы
def convert_to_array(embedding_string):
    try:
        return np.array(ast.literal_eval(embedding_string))
    except Exception as e:
        print(f"Ошибка преобразования: {e}")
        return np.array([])

# Применяем преобразование
df['embeddings'] = df['embeddings'].apply(convert_to_array)

# Удаляем пустые массивы, если это необходимо
df = df[df['embeddings'].apply(lambda x: x.size > 0)]

# Пробуем встраивание массивов в один, если они всех одного размера
try:
    embeddings = np.vstack(df['embeddings'].to_numpy())
except ValueError as e:
    print(f"Ошибка при вертикальном сложении: {e}")

# === Функция для получения эмбеддингов запроса ===
def get_embeddings(query):
    inputs = tokenizer(query, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Усредняем по нужной размерности
    return embeddings.cpu().numpy()


# === Функция рекомендаций через KMeans + KNN ===
def recommend_by_kmeans_knn(query, embeddings, kmeans_model, n_recommendations=10):
    query_embedding = get_embeddings(query)  # Уже numpy
    query_cluster = kmeans_model.predict(query_embedding)[0]  # Получаем кластер
    cluster_indices = np.where(kmeans_model.labels_ == query_cluster)[0]  # Выбираем объекты из кластера

    cluster_embeddings = embeddings[cluster_indices]  # Берем их эмбеддинги

    # Создаем KNN-модель
    nn_model = NearestNeighbors(n_neighbors=n_recommendations, metric='cosine')
    nn_model.fit(cluster_embeddings)

    distances, indices = nn_model.kneighbors(query_embedding)

    recommended_indices = cluster_indices[indices.flatten()]
    return recommended_indices


# === Функция получения экспонатов по индексам ===
def get_exhibits_by_indices(indices):
    return df.iloc[indices][["catalog_num", "title", "author", "date_category"]].to_dict(orient="records")


# === Пример использования ===
user_query = "Хочу посмотреть изображение мужчины в историческом костюме"
recommended_indices = recommend_by_kmeans_knn(user_query, embeddings, kmeans_model)

# Получаем рекомендованные экспонаты
recommended_exhibits = get_exhibits_by_indices(recommended_indices)
print(recommended_exhibits, sep='\n')

In [ ]:
!pip install pgvector

# последняя версия с БД

In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import joblib
from sklearn.neighbors import NearestNeighbors
from transformers import BertTokenizer, BertModel
from dotenv import load_dotenv
from sqlalchemy import create_engine
from pgvector.sqlalchemy import Vector
from sqlalchemy.orm import sessionmaker

In [ ]:
# === Загрузка переменных окружения ===
load_dotenv()

db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')

DATABASE_URL = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Создание подключения к БД
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# === Загрузка модели KMeans ===
kmeans_model = joblib.load("./data-copy/kmeans_model2.joblib")

# === Загрузка модели RuBERT ===
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
# === Функция для получения эмбеддингов запроса ===
def get_embeddings(query):
    inputs = tokenizer(query, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()

In [ ]:
# === Загрузка эмбеддингов из PostgreSQL ===
def load_embeddings():
    query = "SELECT catalog_num, title, author, date_category, embeddings FROM images"
    df = pd.read_sql(query, engine)
    df['embeddings'] = df['embeddings'].apply(lambda x: np.array(x))
    return df

df = load_embeddings()
embeddings = np.vstack(df["embeddings"].values)

In [ ]:
# === Функция рекомендаций через KMeans + KNN без использования эмбеддингов ===
def recommend_by_kmeans_knn(query, kmeans_model, n_recommendations=5):
    # Получаем эмбеддинг запроса
    query_embedding = get_embeddings(query)
    query_embedding = query_embedding.astype(np.float32)
    # Определяем кластер для запроса
    query_cluster = kmeans_model.predict(query_embedding)[0]

    # Получаем индексы объектов в этом кластере
    query_cluster_indices = np.where(kmeans_model.labels_ == query_cluster)[0]

    # Строим модель KNN для ближайших соседей по кластеру (не загружая все эмбеддинги в память)
    nn_model = NearestNeighbors(n_neighbors=n_recommendations, metric='cosine')

    # Вместо загрузки всех эмбеддингов, подгружаем только те, что принадлежат кластеру
    embeddings_query_cluster = get_embeddings_for_indices(query_cluster_indices)

    nn_model.fit(embeddings_query_cluster)

    # Находим ближайших соседей
    distances, indices = nn_model.kneighbors(query_embedding)

    # Получаем индексы ближайших объектов в кластере
    recommended_indices = query_cluster_indices[indices.flatten()]

    return recommended_indices

In [ ]:
# Функция для получения эмбеддингов для выбранных индексов
def get_embeddings_for_indices(indices):
    # Получаем эмбеддинги из БД для объектов с данными индексами
    query = f"SELECT embeddings FROM images WHERE id IN ({','.join(map(str, indices))})"
    df = pd.read_sql(query, engine)

    # Преобразуем эмбеддинги в массив
    embeddings = np.vstack(df["embeddings"].apply(lambda x: np.array(x)).values)

    return embeddings

In [ ]:
# === Функция получения экспонатов по индексам ===
def get_exhibits_by_indices(indices):
    return df.iloc[indices][["catalog_num", "title", "author", "date_category"]].to_dict(orient="records")

In [ ]:
# === Пример использования ===
user_query = "картина, изображающая пейзаж с озером"
recommended_indices = recommend_by_kmeans_knn(user_query, kmeans_model)
recommended_exhibits = get_exhibits_by_indices(recommended_indices)

recommended_exhibits

# вариант 2

In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import joblib
from sklearn.neighbors import NearestNeighbors
from transformers import BertTokenizer, BertModel
from dotenv import load_dotenv
from sqlalchemy import create_engine
from pgvector.sqlalchemy import Vector
from sqlalchemy.orm import sessionmaker
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [ ]:
# === Загрузка переменных окружения ===
load_dotenv()

db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')

DATABASE_URL = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Создание подключения к БД
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# === Загрузка модели KMeans ===
kmeans_model = joblib.load("./data-copy/kmeans_model2.joblib")

# === Загрузка модели RuBERT ===
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [ ]:
# === Загрузка модели Spacy для лемматизации ===
nlp = spacy.load('ru_core_news_sm')

# === Список стоп-слов для TF-IDF ===
STOP_WORDS = nlp.Defaults.stop_words

In [ ]:
# === Функция для лемматизации и очистки текста ===
def lemmatize_text_spacy(text):
    doc = nlp(text.lower())  # Приводим к нижнему регистру
    lemmatized_words = [
        token.lemma_ for token in doc
        if token.is_alpha  # Убираем числа и знаки препинания
        and len(token.lemma_) > 1  # Исключаем отдельные буквы
        and token.lemma_ not in STOP_WORDS  # Исключаем стоп-слова
    ]
    return ' '.join(lemmatized_words)

In [ ]:
# === Функция для получения эмбеддингов запроса ===
def get_embeddings(query):
    inputs = tokenizer(query, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()

In [ ]:
# === Загрузка эмбеддингов из PostgreSQL ===
def load_embeddings():
    query = "SELECT catalog_num, title, author, date_category, embeddings FROM images"
    df = pd.read_sql(query, engine)
    df['embeddings'] = df['embeddings'].apply(lambda x: np.array(x))
    return df

In [ ]:
df = load_embeddings()
embeddings = np.vstack(df["embeddings"].values)

In [ ]:
# === Функция для извлечения ключевых слов с использованием TF-IDF ===
def extract_keywords_with_tfidf(df, top_n=15):
    # Объединяем описание и название в единый текст и лемматизируем
    df['combined_text'] = df['description'] + ' ' + df['title']
    df['lemmatized_text'] = df['combined_text'].apply(lemmatize_text_spacy)

    # Инициализируем TF-IDF векторизатор с русскими стоп-словами
    vectorizer = TfidfVectorizer(stop_words=list(STOP_WORDS))
    tfidf_matrix = vectorizer.fit_transform(df['lemmatized_text'])
    feature_names = vectorizer.get_feature_names_out()

    df['key_words'] = ''

    for index in tqdm(range(len(df)), total=len(df), desc="Processing rows", ncols=100):
        # Получаем TF-IDF веса для текущей строки
        tfidf_vector = tfidf_matrix[index].toarray().flatten()
        top_indices = tfidf_vector.argsort()[-top_n:][::-1]

        # Извлекаем ключевые слова
        key_words = [feature_names[i] for i in top_indices if tfidf_vector[i] > 0]

        # Сохраняем ключевые слова
        df.at[index, 'key_words'] = ','.join(key_words)

    # Удаляем временные колонки
    df.drop(columns=['combined_text', 'lemmatized_text'], inplace=True)

    return df

In [ ]:
# === Функция рекомендаций через KMeans + KNN ===
def recommend_by_kmeans_knn(query, embeddings, kmeans_model, n_recommendations=5):
    # Преобрабатываем запрос
    preprocessed_query = lemmatize_text_spacy(query)

    # Получаем эмбеддинг для обработанного запроса
    query_embedding = get_embeddings([preprocessed_query])

    # Определяем кластер запроса
    query_cluster = kmeans_model.predict(query_embedding)[0]

    # Находим индексы экспонатов из этого кластера
    cluster_indices = np.where(kmeans_model.labels_ == query_cluster)[0]
    cluster_embeddings = embeddings[cluster_indices]

    # Используем KNN для поиска ближайших экспонатов
    nn_model = NearestNeighbors(n_neighbors=n_recommendations, metric='cosine')
    nn_model.fit(cluster_embeddings)

    distances, indices = nn_model.kneighbors(query_embedding)
    recommended_indices = cluster_indices[indices.flatten()]

    return recommended_indices

In [ ]:
# === Функция получения экспонатов по индексам ===
def get_exhibits_by_indices(indices):
    return df.iloc[indices][["catalog_num", "title", "author", "date_category"]].to_dict(orient="records")

In [ ]:
# === Пример использования ===
user_query = "картина, изображающая пейзаж с озером"
recommended_indices = recommend_by_kmeans_knn(user_query, embeddings, kmeans_model)
recommended_exhibits = get_exhibits_by_indices(recommended_indices)

print(recommended_exhibits)

In [ ]:
# === Пример использования ===
user_query = "Хочу посмотреть изображение мужчины в историческом костюме."
recommended_indices = recommend_by_kmeans_knn(user_query, embeddings, kmeans_model)
recommended_exhibits = get_exhibits_by_indices(recommended_indices)

print(recommended_exhibits)
